<a href="https://colab.research.google.com/github/HIsu1231/AI_INOVATION_SQUARE/blob/master/tensorflow_diabetes_NN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 26kB/s 
     |████████████████████████████████| 512kB 41.1MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=6e50a9860c10f811f00f08fc763a80a03196e4c18fe72e330cc7e269bdf7e637
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninsta

In [3]:
import numpy as np
import tensorflow as tf

In [8]:
class DataGeneration():
    
    def __init__(self, file_path, seperation_rate, target_position):
        
        self.file_path = file_path

        self.seperation_rate = seperation_rate

        if target_position == -1 or target_position == 0:
            self.target_position = target_position
        else:
            str_err = 'target_position must be -1 or 0'
            raise Exception(str_err)

    def __display_target_distribution(self, data, str_of_kind):

        target_data = data[:, self.target_position]

        unique, counts = np.unique(target_data, return_counts=True)

        unique_target = []

        print("=================================================================================================")
        for i in range(len(unique)):
            print("[DataGeneration] unique number of " + str_of_kind + " = ", unique[i], ", count = ", counts[i])
            unique_target.append(unique[i])

            print("[DataGeneration] unique number of " + str_of_kind + " = ", unique_target[i], ", ratio = ", np.round(100 * len(unique_target) / len(target_data),2) ,'%')
        print("=================================================================================================")

    def generate(self):

        try:
            loaded_data = np.loadtxt(self.file_path, delimiter=',')

        except Exception as err:
          print("[DataGeneration::generate()] ", str(err))
          raise Exception(str(err))

        print("loaded_data.shape = ", loaded_data.shape)

        self.__display_target_distribution(loaded_data, 'original data')

        total_data_num = len(loaded_data)
        test_data_num = int(total_data_num * self.seperation_rate)

        np.random.shuffle(loaded_data)

        test_data = loaded_data[0:test_data_num]
        training_data = loaded_data[test_data_num: ]

        self.__display_target_distribution(training_data, "training data")
        self.__display_target_distribution(test_data, "test data")

        return training_data, test_data

In [10]:
file_path = './drive/MyDrive/AI_INOVATION_SQUARE/data/(200309)diabetes.csv'
seperation_rate = 0.2
target_position = -1

data_obj = DataGeneration(file_path, seperation_rate, target_position)

(training_data, test_data) = data_obj.generate()

print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

loaded_data.shape =  (759, 9)
[DataGeneration] unique number of original data =  0.0 , count =  263
[DataGeneration] unique number of original data =  0.0 , ratio =  0.13 %
[DataGeneration] unique number of original data =  1.0 , count =  496
[DataGeneration] unique number of original data =  1.0 , ratio =  0.26 %
[DataGeneration] unique number of training data =  0.0 , count =  213
[DataGeneration] unique number of training data =  0.0 , ratio =  0.16 %
[DataGeneration] unique number of training data =  1.0 , count =  395
[DataGeneration] unique number of training data =  1.0 , ratio =  0.33 %
[DataGeneration] unique number of test data =  0.0 , count =  50
[DataGeneration] unique number of test data =  0.0 , ratio =  0.66 %
[DataGeneration] unique number of test data =  1.0 , count =  101
[DataGeneration] unique number of test data =  1.0 , ratio =  1.32 %
training_data.shape =  (608, 9)
test_data.shape =  (151, 9)


In [11]:
training_x_data = training_data[:, 0:-1]
training_t_data = training_data[:, [-1]]

print("training_x_data.shape = ", training_x_data.shape)
print("training_t_data.shape = ", training_t_data.shape)

test_x_data = test_data[:, 0:-1]
test_t_data = test_data[:, [-1]]

print("test_x_data.shape = ", test_x_data.shape)
print("test_t_data.shape = ", test_t_data.shape)

training_x_data.shape =  (608, 8)
training_t_data.shape =  (608, 1)
test_x_data.shape =  (151, 8)
test_t_data.shape =  (151, 1)


In [24]:
input_nodes = training_x_data.shape[1]
hidden1_nodes = 4
hidden2_nodes = 4
output_nodes = 1

X = tf.placeholder(tf.float32, [None,input_nodes])
T = tf.placeholder(tf.float32, [None,output_nodes])

W2 = tf.Variable(tf.random_normal([input_nodes,hidden1_nodes]))
b2 = tf.Variable(tf.random_normal([hidden1_nodes]))

W3 = tf.Variable(tf.random_normal([hidden1_nodes, hidden2_nodes]))
b3 = tf.Variable(tf.random_normal([hidden2_nodes]))

W4 = tf.Variable(tf.random_normal([hidden2_nodes, output_nodes]))
b4 = tf.Variable(tf.random_normal([output_nodes]))

In [29]:
z2 = tf.matmul(X,W2) + b2
a2 = tf.sigmoid(z2)

z3 = tf.matmul(a2,W3) + b3
a3 = tf.sigmoid(z3)

z4 = tf.matmul(a3,W4) + b4
y = tf.sigmoid(z4)

loss = -tf.reduce_mean(T * tf.log(y) + (1-T) * tf.log(1-y))

In [30]:
learning_rate = 1e-1

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [31]:
predicted = tf.cast(y > 0.5, tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(T,predicted),tf.float32))

In [32]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for i in range(20001):

        loss_val, _ = sess.run([loss, train], feed_dict={X: training_x_data, T: training_t_data})

        if i % 1000 == 0:
            print("i = ", i, ", loss value = ", loss_val)

    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X: test_x_data, T: test_t_data})

    print("\ny_val.shape = ", y_val.shape, ", predicted_val.shape = ", predicted_val.shape)
    print("\nAccuracy = ", accuracy_val)

i =  0 , loss value =  0.7892712
i =  1000 , loss value =  0.6438896
i =  2000 , loss value =  0.59321153
i =  3000 , loss value =  0.4906034
i =  4000 , loss value =  0.46600562
i =  5000 , loss value =  0.46019584
i =  6000 , loss value =  0.45754823
i =  7000 , loss value =  0.45574358
i =  8000 , loss value =  0.45417395
i =  9000 , loss value =  0.45258704
i =  10000 , loss value =  0.4508401
i =  11000 , loss value =  0.44882092
i =  12000 , loss value =  0.4464018
i =  13000 , loss value =  0.44340125
i =  14000 , loss value =  0.43963823
i =  15000 , loss value =  0.43523145
i =  16000 , loss value =  0.4308088
i =  17000 , loss value =  0.42698082
i =  18000 , loss value =  0.42384398
i =  19000 , loss value =  0.42124513
i =  20000 , loss value =  0.41905442

y_val.shape =  (151, 1) , predicted_val.shape =  (151, 1)

Accuracy =  0.78807944
